# 프로젝트 진행 과정
 * 우선 분석할 수 있는 형태로 가공하는 데이터 전처리
 * 토크나이저 생성
 * 데이터 준비 및 분리
 * 하이퍼 파라미터 설정
 * 모델만들기
 * 모델 훈련시키기
 * 문장 디코딩 함수 정의 후 문장출력
 * 하이퍼 파라미터 조정으로 성능개선


# 필요 패키지 로드

In [1]:
import re                  # 정규표현식을 위한 Regex 지원 모듈 (문장 데이터를 정돈하기 위해) 
import numpy as np         # 변환된 문장 데이터(행렬)을 편하게 처리하기 위해
import tensorflow as tf    # 대망의 텐서플로우!
import os

# 데이터 읽어오기

In [2]:
import glob # 경로안에 모든 파일이나 디렉토리명을 리스트로 뽑을 수 있음
import os

txt_file_path = os.getenv('HOME')+'/aiffel/lyricist/data/lyrics/*'

txt_list = glob.glob(txt_file_path)
raw_corpus = []
# 여러개의 txt 파일을 모두 읽어서 raw_corpus 에 담습니다.
for txt_file in txt_list:
    with open(txt_file, "r") as f: 
        raw = f.read().splitlines() # 파일을 읽기전용으로 얽이서
        raw_corpus.extend(raw) #raw_corpus list에 요소(문장)으로 추가함(말뭉치)
# 만들어진 말뭉치를 확인합니다.
print("데이터 크기:", len(raw_corpus))
print("Examples:\n", raw_corpus[:10])

데이터 크기: 187088
Examples:
 ['Come on, come on', 'You think you drive me crazy', 'Come on, come on', 'You and whose army?', 'You and your cronies', 'Come on, come on', 'Holy Roman empire', 'Come on if you think', 'Come on if you think', 'You can take us on']


# 데이터 정제

## 정규식으로 필요 문자만 추출

In [3]:
def preprocess_sentence(sentence):
    sentence = sentence.lower().strip()       # 소문자로 바꾸고 양쪽 공백을 삭제
  
    # 아래 3단계를 거쳐 sentence는 스페이스 1개를 delimeter로 하는 소문자 단어 시퀀스로 바뀝니다.
    sentence = re.sub(r"([?.!,¿])", r" \1 ", sentence)        # 패턴의 특수문자를 만나면 특수문자 양쪽에 공백을 추가
    sentence = re.sub(r'[" "]+', " ", sentence)                  # 공백 패턴을 만나면 스페이스 1개로 치환
    sentence = re.sub(r"[^a-zA-Z?.!,¿]+", " ", sentence)  # a-zA-Z?.!,¿ 패턴을 제외한 모든 문자(공백문자까지도)를 스페이스 1개로 치환

    sentence = sentence.strip()
    #이런식이면 hi- im 이면 두칸 벌어지지않나?
    sentence = '<start> ' + sentence + ' <end>'      # 이전 스텝에서 본 것처럼 문장 앞뒤로 <start>와 <end>를 단어처럼 붙여 줍니다
    return sentence

print(preprocess_sentence("This @_is ;;;sample        sentence."))   # 이 문장이 어떻게 필터링되는지 확인해 보세요.
print(len(preprocess_sentence("This @_is ;;;sample        sentence.").split(" ")))

<start> this is sample sentence . <end>
7


## 위의 함수를 활용한 전처리 및 문장수를 기준으로 선별

In [4]:
corpus1 = []
# 원본 데이터에서 sentence를 뽑아서
for sentence in raw_corpus:
    if len(sentence) == 0: continue
    #elif len(sentence) > 16: continue # 이걸로 거르면 13200개밖에 안나옴, 이건 문장의 글자 수 기준으로.. 띄어쓰기 기준으로 나눈 단어를 기준으로 해야함
    # sentence가 공백이 아니면 sentence를 인자로 받아 위의 함수를 실행
    #elif len(sentence.split(" ")) > 15: continue
    corpus1.append(preprocess_sentence(sentence))

corpus1[:10]

['<start> come on , come on <end>',
 '<start> you think you drive me crazy <end>',
 '<start> come on , come on <end>',
 '<start> you and whose army ? <end>',
 '<start> you and your cronies <end>',
 '<start> come on , come on <end>',
 '<start> holy roman empire <end>',
 '<start> come on if you think <end>',
 '<start> come on if you think <end>',
 '<start> you can take us on <end>']

## 띄어쓰기를 기준으로, 단어수를 기준으로 문장 분리
 * 단어수 15개 이상인 문장은 제거
 * 너무 많은 padding이 필요하기 때문

In [5]:
corpus = []
for i, j in enumerate(corpus1) :
    if len(j.split(" ")) > 15 : pass
    else : corpus.append(j)
print(corpus[:3])
#복잡해져서..??
#기준은..??데이터수에 비례
#토크나이저를 만들때는 기본적으로 모든 단어를 실험 - 지나치게 많은 시간이 소요됨

['<start> come on , come on <end>', '<start> you think you drive me crazy <end>', '<start> come on , come on <end>']


## 토크나이저 생성

In [6]:
def tokenize(corpus):
    # 텐서플로우에서 제공하는 Tokenizer 패키지를 생성
    tokenizer = tf.keras.preprocessing.text.Tokenizer( # 정제된 데이터를 토큰화하고, 단어 사전을 만들어주며, 데이터를 숫자로 변환까지 한 방에 처리 - 벡터화(vectorize)
        num_words=12000,  # 전체 단어의 개수 - 지나친 연산 예방
        filters=' ',    # 별도로 전처리 로직을 추가할 수 있습니다. 이번에는 사용하지 않겠습니다.
        oov_token="<unk>"  # out-of-vocabulary, 사전에 없었던 단어는 어떤 토큰으로 대체할지
    )
    tokenizer.fit_on_texts(corpus)   # 우리가 구축한 corpus로부터 Tokenizer가 사전을 자동구축하게 됩니다.

    # 이후 tokenizer를 활용하여 모델에 입력할 데이터셋을 구축하게 됩니다.
    tensor = tokenizer.texts_to_sequences(corpus)   # tokenizer는 구축한 사전으로부터 corpus를 해석해 Tensor로 변환합니다.

    # 입력 데이터의 시퀀스 길이를 일정하게 맞추기 위한 padding  메소드를 제공합니다.
    # maxlen의 디폴트값은 None입니다. 이 경우 corpus의 가장 긴 문장을 기준으로 시퀀스 길이가 맞춰집니다.
    tensor = tf.keras.preprocessing.sequence.pad_sequences(tensor, padding='post', maxlen = 15)  

    print(tensor,tokenizer)
    return tensor, tokenizer

tensor, tokenizer = tokenize(corpus)

[[  2  68  18 ...   0   0   0]
 [  2   7 130 ...   0   0   0]
 [  2  68  18 ...   0   0   0]
 ...
 [  2  47  47 ...   0   0   0]
 [  2   4  24 ...   0   0   0]
 [  2  47  47 ...   0   0   0]] <keras_preprocessing.text.Tokenizer object at 0x7f66c11ee8d0>


## 15개만 추출되는것 확인

In [7]:
#확인
print(tensor[:3, :])

[[  2  68  18   5  68  18   3   0   0   0   0   0   0   0   0]
 [  2   7 130   7 570  12 273   3   0   0   0   0   0   0   0]
 [  2  68  18   5  68  18   3   0   0   0   0   0   0   0   0]]


## index 10개만 추출

In [8]:
for idx in tokenizer.index_word:
    print(idx, ":", tokenizer.index_word[idx])

    if idx >= 10: break

1 : <unk>
2 : <start>
3 : <end>
4 : i
5 : ,
6 : the
7 : you
8 : and
9 : a
10 : to


# 평가 데이터셋 분리

훈련 데이터와 평가 데이터를 분리하세요!

tokenize() 함수로 데이터를 Tensor로 변환한 후, sklearn 모듈의 train_test_split() 함수를 사용해 훈련 데이터와 평가 데이터를 분리하도록 하겠습니다. 단어장의 크기는 12,000 이상으로 설정하세요! 총 데이터의 20%를 평가 데이터셋으로 사용해 주세요!

 * tensor를 소스와 타겟으로 분리
 * padding(post) 확인

In [9]:
#정확히 어떤 느낌의 슬라이싱인지 모르겠다. 차원이 들어가는건가? 한번 더 확인할 것!
# 토크나이저가 corpus를 텐서(숫자로 변환된 데이터)로 전환

src_input = tensor[:, :-1]  # tensor에서 마지막 토큰을 잘라내서 소스 문장을 생성합니다. 마지막 토큰은 <end>가 아니라 <pad>일 가능성이 높습니다.
tgt_input = tensor[:, 1:]    # tensor에서 <start>를 잘라내서 타겟 문장을 생성합니다.

print(src_input[0])
print(tgt_input[0])

#15개로 잘랐는데 앞뒤로 하나씩 제거하니 14개가 남음

[ 2 68 18  5 68 18  3  0  0  0  0  0  0  0]
[68 18  5 68 18  3  0  0  0  0  0  0  0  0]


In [10]:
import numpy as np
from sklearn.model_selection import train_test_split
enc_train, enc_val, dec_train, dec_val = train_test_split(src_input,
                                                         tgt_input,
                                                         test_size = 0.2,
                                                         random_state = 32)
#여기까지 올바르게 진행했을 경우, 아래 실행 결과를 확인할 수 있습니다.

print("Source Train:", enc_train.shape)
print("Target Train:", dec_train.shape)
print("Source test:", enc_val.shape)
print("Target test:", dec_val.shape)

# out: (데이터 갯수, 구성요소 갯수)
# Source Train: (124960, 14)
# Target Train: (124960, 14)

Source Train: (124981, 14)
Target Train: (124981, 14)
Source test: (31246, 14)
Target test: (31246, 14)


## 하이퍼 파라미터 설정

In [11]:
BUFFER_SIZE = len(enc_train)
BATCH_SIZE = 256 # 커지면 램을 많이 먹는 대신 epoch당 연산횟수가 줄어듬 / 작아지면 시간이 오래걸림
steps_per_epoch = len(enc_train) // BATCH_SIZE # 전체 데이터를 배치사이즈로 나누면 epoch당 연산횟수가 나옴

VOCAB_SIZE = tokenizer.num_words + 1    # tokenizer가 구축한 단어사전 내 12,000개와, 여기 포함되지 않은 0:<pad>를 포함하여 12,001개

## 데이터셋 변형, 셔플(효과적인 학습을 위해)

In [12]:
dataset = tf.data.Dataset.from_tensor_slices((enc_train, dec_train)).shuffle(BUFFER_SIZE)
dataset = dataset.batch(BATCH_SIZE, drop_remainder=True)
dataset

<BatchDataset shapes: ((256, 14), (256, 14)), types: (tf.int32, tf.int32)>

## validation set 구성

In [13]:
dataset_val = tf.data.Dataset.from_tensor_slices((enc_val, dec_val)).shuffle(BUFFER_SIZE)
dataset_val = dataset_val.batch(BATCH_SIZE, drop_remainder=True)
dataset_val

<BatchDataset shapes: ((256, 14), (256, 14)), types: (tf.int32, tf.int32)>

# Step 5. 인공지능 만들기

데이터가 커서 훈련하는 데 시간이 제법 걸릴 겁니다. 여유를 가지고 작업하시면 좋아요 :)

In [14]:
class TextGenerator(tf.keras.Model):
    def __init__(self, vocab_size, embedding_size, hidden_size):
        super(TextGenerator, self).__init__()
        
        self.embedding = tf.keras.layers.Embedding(vocab_size, embedding_size)
        self.rnn_1 = tf.keras.layers.LSTM(hidden_size, return_sequences=True)
        self.rnn_2 = tf.keras.layers.LSTM(hidden_size, return_sequences=True)
        self.linear = tf.keras.layers.Dense(vocab_size)
        
    def call(self, x):
        out = self.embedding(x)
        out = self.rnn_1(out)
        out = self.rnn_2(out)
        out = self.linear(out)
        
        return out
    
embedding_size = 128
hidden_size = 2048
model = TextGenerator(tokenizer.num_words + 1, embedding_size , hidden_size)

In [15]:
for src_sample, tgt_sample in dataset.take(1): break
model(src_sample)

<tf.Tensor: shape=(256, 14, 12001), dtype=float32, numpy=
array([[[ 7.44041290e-06, -8.54541577e-05,  6.44726024e-05, ...,
          8.26207906e-06, -6.34825556e-05,  1.34171743e-04],
        [ 5.36844636e-05, -1.01988873e-04,  1.04880171e-04, ...,
         -4.75803608e-05, -1.41333646e-06,  2.08675076e-04],
        [ 9.98819742e-05, -1.33002075e-04,  3.48095023e-06, ...,
         -1.86793171e-04,  1.99035043e-04,  2.38055873e-04],
        ...,
        [-5.49794233e-04,  2.42343801e-03,  6.96828880e-04, ...,
         -2.51508260e-04, -1.10568781e-03, -4.86383040e-04],
        [-6.88098429e-04,  2.78599467e-03,  7.97803747e-04, ...,
         -3.31657298e-04, -1.43381802e-03, -6.36837678e-04],
        [-8.03675794e-04,  3.09268362e-03,  8.80414853e-04, ...,
         -4.19336866e-04, -1.74940540e-03, -7.66353623e-04]],

       [[ 7.44041290e-06, -8.54541577e-05,  6.44726024e-05, ...,
          8.26207906e-06, -6.34825556e-05,  1.34171743e-04],
        [-2.75006605e-04, -1.54493755e-04, -7

In [16]:
model.summary()

Model: "text_generator"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding (Embedding)        multiple                  1536128   
_________________________________________________________________
lstm (LSTM)                  multiple                  17833984  
_________________________________________________________________
lstm_1 (LSTM)                multiple                  33562624  
_________________________________________________________________
dense (Dense)                multiple                  24590049  
Total params: 77,522,785
Trainable params: 77,522,785
Non-trainable params: 0
_________________________________________________________________


In [17]:
optimizer = tf.keras.optimizers.Adam()
loss = tf.keras.losses.SparseCategoricalCrossentropy(
    from_logits=True,
    reduction='none'
)

model.compile(loss=loss, optimizer=optimizer)
model.fit(dataset, validation_data = dataset_val,epochs=10)
#30회면 loss가 1.1까지 떨어짐


Epoch 1/10
488/488 [==============================] - 176s 360ms/step - loss: 3.4814 - val_loss: 3.0498
Epoch 2/10
488/488 [==============================] - 178s 364ms/step - loss: 2.9096 - val_loss: 2.8225
Epoch 3/10
488/488 [==============================] - 179s 366ms/step - loss: 2.6620 - val_loss: 2.6623
Epoch 4/10
488/488 [==============================] - 179s 367ms/step - loss: 2.4206 - val_loss: 2.5308
Epoch 5/10
488/488 [==============================] - 179s 367ms/step - loss: 2.1792 - val_loss: 2.4241
Epoch 6/10
488/488 [==============================] - 180s 369ms/step - loss: 1.9477 - val_loss: 2.3413
Epoch 7/10
488/488 [==============================] - 179s 366ms/step - loss: 1.7334 - val_loss: 2.2816
Epoch 8/10
488/488 [==============================] - 178s 366ms/step - loss: 1.5441 - val_loss: 2.2382
Epoch 9/10
488/488 [==============================] - 177s 364ms/step - loss: 1.3820 - val_loss: 2.2156
Epoch 10/10
488/488 [==============================] - 177s 363m

In [18]:
def generate_text(model, tokenizer, init_sentence="<start>", max_len=20):
    # 테스트를 위해서 입력받은 init_sentence도 일단 텐서로 변환합니다.
    test_input = tokenizer.texts_to_sequences([init_sentence])
    test_tensor = tf.convert_to_tensor(test_input, dtype=tf.int64)
    end_token = tokenizer.word_index["<end>"]

    # 텍스트를 실제로 생성할때는 루프를 돌면서 단어 하나씩 생성해야 합니다. 
    while True:
        predict = model(test_tensor)  # 입력받은 문장의 텐서를 입력합니다. 
        predict_word = tf.argmax(tf.nn.softmax(predict, axis=-1), axis=-1)[:, -1]   # 우리 모델이 예측한 마지막 단어가 바로 새롭게 생성한 단어가 됩니다. 

        # 우리 모델이 새롭게 예측한 단어를 입력 문장의 뒤에 붙여 줍니다. 
        test_tensor = tf.concat([test_tensor,
                                 tf.expand_dims(predict_word, axis=0)], axis=-1)

        # 우리 모델이 <end>를 예측했거나, max_len에 도달하지 않았다면  while 루프를 또 돌면서 다음 단어를 예측해야 합니다.
        if predict_word.numpy()[0] == end_token: break
        if test_tensor.shape[1] >= max_len: break

    generated = ""
    # 생성된 tensor 안에 있는 word index를 tokenizer.index_word 사전을 통해 실제 단어로 하나씩 변환합니다. 
    for word_index in test_tensor[0].numpy():
        generated += tokenizer.index_word[word_index] + " "

    return generated   # 이것이 최종적으로 모델이 생성한 자연어 문장입니다.

In [19]:
#Loss

loss = tf.keras.losses.SparseCategoricalCrossentropy(
    from_logits=True, reduction='none')
print(loss)

In [20]:
generate_text(model, tokenizer, init_sentence="<start> i love", max_len=20)

'<start> i love you , i m not gonna crack <end> '

In [27]:
generate_text(model, tokenizer, init_sentence="<start> here i", max_len=20)

'<start> here i am , baby . it always ends up this way , <end> '

# 회고

시작부터 재미있었습니다. 극대본을 학습시키니 극대본풍의 대사를 반환하다니... 이제까지 노드중에 가장 결과물이 기대된 노드가 아니었나 싶습니다. 15개 단어를 초과하는 문장을 제거하는 부분에서 시간이 좀 걸렸고, 아래 함수를 정의한 부분은 이해하지 못했지만, 결과물로 나온 모델이 뱉어내는 가사들을 보고 학습시키며 즐거웠습니다.(이제까지의 노드들 중 가장 인공지능스럽지 않았나..싶습니다.)
다소 힙한 모델이 완성되었습니다. 'that s true i got a big bitch'라거나, 

이번 노드에서 가장 중요한 개념은 다음의 두 하이퍼 파라미터로 보입니다. embedding_size와 hidden_size입니다.
 * Embedding_size는 워드 벡터의 차원수, 즉 단어가 추상적으로 표현되는 크기입니다. Embedding으로 단어를 추상적으로 변환(특징을 수치로 나타냄)할 수 있으며, 단위가 커질 수록 던어의 특징을 잘 잡아내 말뭉치를 보다 풍부하게 표현할 수 있지만, 충분한 데이터가 주어지지 않는다면 오히려 학습을 방해하는 원인이 될 수 있습니다.
 * Hidden_size도 유사한 맥락입니다. LSTM 레이어의 hidden state 의 차원수로, 얼마나 많은 일꾼이 특징을 잡아내기 위해 노력하는지로 생각해도 된다고 합니다.(layer의 뉴런을 말하는 걸까요?)

이번 노드의 평가 기준을 맞추기 위해 조작한것도 이 두 하이퍼 파라미터였습니다. hidden_size를 늘리면 연산이 증가해 시간이 오래 걸릴것 같아 우선 embedding_size를 조절했습니다. 기존 256(validation_loss = 2.52)을 기준으로, 128로 줄였을 때가 가장 높은 성능을 보였습니다.(validation_loss = 2.49)64로 줄였을때는 오히려 성능이 감소했습니다(val_loss = 2.63). 기존 노드의 1024를 기준으로 512로 줄였을 때 성능이 감소하였으며 2048로 늘렸을 때 2.208로 성능이 가장 좋았습니다. 이처럼 여러가지 수치를 조합해본 결과, embedding_size = 128, hidden_size는 2048일 때 가장 높은 성능을 보였습니다(validation loss = 2.208). hidden_size를 늘렸을 때 성능이 높아지는것을 확인해 4096까지 늘려보았지만, 노트북이 지속적으로 멈춰 2048이상으로는 실험하지 못했습니다.
 
이미지는 시각적으로 느껴지는 즐거움이 있는 반면, 텍스트는 보다 깊은 부분까지 생각해볼 수 있는 분야인것 같습니다. 프로젝트를 거치면서, 보다 높은 수준의 모델을 만들어보고 싶어지는 노드였습니다.